# Experiment of editing the graph.

In [1]:
import os
import tensorflow as tf
import tensorflow.contrib.graph_editor as ge

/home/holmescn/.pyenv/versions/anaconda3-5.2.0/envs/tensorflow-gpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Build basic graph
os.system('rm -rf /tmp/graph')

g = tf.Graph()

with g.as_default():
    a = tf.placeholder(tf.float32, shape=(), name='a')
    b = tf.placeholder(tf.float32, shape=(), name='b')
    c = a + b
    d = tf.identity(c)
    e = tf.identity(c)

c1 = tf.identity(c)

ge.connect(c1.op, d.op)
ge.connect(c1.op, e.op)

tf.summary.FileWriter("/tmp/graph", g).close()

In [75]:
# Simple CNN block
os.system('rm -rf /tmp/graph')

g = tf.Graph()

with g.as_default():
    inputs = tf.placeholder(dtype=tf.float32, shape=(None, 32, 32, 3), name='inputs')
    conv1 = tf.layers.conv2d(inputs, 32, 3, padding='same', activation=tf.nn.relu, name='conv1')
    conv2 = tf.layers.conv2d(conv1, 10, 3, padding='same', activation=tf.nn.relu, name='conv2')
    pool = tf.layers.average_pooling2d(conv2, 32, 1, name='pool')
    flatten = tf.layers.flatten(pool)
    logits = tf.layers.dense(flatten, units=10)
    
    labels = tf.placeholder(dtype=tf.int32, shape=(None, 1))
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train_op = optimizer.minimize(loss)

conv1_1 = tf.identity(conv1)

for consumer in conv1.consumers():
    for index, tensor in enumerate(consumer.inputs):
        if tensor == conv1 and consumer != conv1_1.op:
            consumer._update_input(index, conv1_1, False)


tf.summary.FileWriter("/tmp/graph", g).close()